# Bootcamp exercise: Timeseries analysis with analysis tools

**Description:** Introduction to timeseries analysis on DRP difference imaging products

**Contact authors:** Eric Bellm

**Last verified to run:** 

**LSST Science Piplines version:** 



Check the version of the stack you are using

In [1]:
!eups list -s | grep lsst_distrib

lsst_distrib          gdf42428520+aa7779d39a 	current d_2023_05_03 setup


Are you using a local version of `analysis_tools`?

In [2]:
import lsst.analysis.tools
print(lsst.analysis.tools.__file__)

/sdf/group/rubin/u/ebellm/stack/analysis_tools/python/lsst/analysis/tools/__init__.py


## Preliminaries

In [3]:
import numpy as np
import lsst.daf.butler as dafButler

In [4]:
# Point to existing sandbox repo if you prefer to skip processing steps
#collections = ['u/bechtol']
#repo = '/sdf/group/rubin/user/bechtol/bootcamp_2023/rc2_subset/SMALL_HSC/'

collections = ['HSC/runs/RC2/w_2023_07/DM-38042/20230308T213613Z']
repo = '/repo/main/'


# User instance of the repo if you have processed rc2_subset yourself
#collections = ['u/%s'%os.environ['USER']]
#repo = '/sdf/group/rubin/user/%s/bootcamp_2023/rc2_subset/SMALL_HSC/'%(os.environ['USER'])

In [5]:
butler = dafButler.Butler(repo, collections=collections)
registry = butler.registry

Check what (tabular) dataset types are present in the collection.  We are going to work with pre-associated DIASources, which are not always present.

In [5]:
required_dataset_type = 'diaSourceTable_tract'
has_required_dataset_type = False

for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collections).any(execute=False, exact=False):
        if datasetType.storageClass_name == 'DataFrame':
            print(datasetType)
        if datasetType.name == required_dataset_type:
            has_required_dataset_type = True

DatasetType('goodSeeingDiff_assocDiaSrcTable', {skymap, tract, patch}, DataFrame)
DatasetType('goodSeeingDiff_diaObjTable', {skymap, tract, patch}, DataFrame)
DatasetType('goodSeeingDiff_fullDiaObjTable', {skymap, tract, patch}, DataFrame)
DatasetType('diaSourceTable_tract', {skymap, tract}, DataFrame)
DatasetType('diaObjectTable_tract', {skymap, tract}, DataFrame)
DatasetType('forcedSourceOnDiaObjectTable', {skymap, tract, patch}, DataFrame)
DatasetType('forcedSourceOnDiaObjectTable_tract', {skymap, tract}, DataFrame)
DatasetType('forcedSourceTable_tract', {skymap, tract}, DataFrame)
DatasetType('forcedSourceTable', {skymap, tract, patch}, DataFrame)
DatasetType('mergedForcedSourceOnDiaObject', {band, instrument, skymap, detector, physical_filter, tract, visit}, DataFrame)


In [6]:
if not has_required_dataset_type:
    raise ValueError(f'Required dataset type {required_dataset_type} not present in collections {collections} and repo {repo}!')

If the cell above raises an error, you will need a different dataset!

## Object tables

In [6]:
refs = sorted(registry.queryDatasets("diaObjectTable_tract"))
print(len(refs))

3


In [7]:
refs[0].dataId

{skymap: 'hsc_rings_v1', tract: 9615}

In [8]:
objTable = butler.get(refs[0])
# how to select subsets of columns
#objTable = butler.get(refs[0],parameters={'columns': ['ra','decl']})
objTable

,ra,decl,nDiaSources,radecTai,gPSFluxLinearSlope,gPSFluxLinearIntercept,gPSFluxMAD,gPSFluxMaxSlope,gPSFluxErrMean,gPSFluxMean,...,yPSFluxPercentile05,yPSFluxPercentile25,yPSFluxPercentile50,yPSFluxPercentile75,yPSFluxPercentile95,yPSFluxSigma,yTOTFluxSigma,yPSFluxSkew,yPSFluxChi2,yPSFluxStetsonJ
diaObjectId,,,,,,,,,,,,,,,,,,,,,
3425264593545461761,217.080213,-0.064201,1,56741.506686,NaN,NaN,NaN,NaN,NaN,NaN,...,-2628.276984,-2628.276984,-2628.276984,-2628.276984,-2628.276984,NaN,NaN,NaN,1.301390e-30,NaN
3425264593545461762,217.080244,-0.069648,1,56741.506686,NaN,NaN,NaN,NaN,NaN,NaN,...,-2140.304091,-2140.304091,-2140.304091,-2140.304091,-2140.304091,NaN,NaN,NaN,0.000000e+00,NaN
3425264593545461763,217.079951,-0.068807,1,56741.506686,NaN,NaN,NaN,NaN,NaN,NaN,...,-2636.178480,-2636.178480,-2636.178480,-2636.178480,-2636.178480,NaN,NaN,NaN,0.000000e+00,NaN
3425264593545461764,217.079667,-0.058519,1,56741.506686,NaN,NaN,NaN,NaN,NaN,NaN,...,-2449.343813,-2449.343813,-2449.343813,-2449.343813,-2449.343813,NaN,NaN,NaN,0.000000e+00,NaN
3425264593545461765,217.079072,-0.049209,1,56741.506686,NaN,NaN,NaN,NaN,NaN,NaN,...,-2099.895280,-2099.895280,-2099.895280,-2099.895280,-2099.895280,NaN,NaN,NaN,0.000000e+00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425616437266356022,215.642458,1.494351,1,57110.598862,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3425616437266356023,215.642434,1.511678,1,57110.598862,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3425616437266356024,215.641859,1.513705,1,57110.598862,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
objTable.columns.values

array(['ra', 'decl', 'nDiaSources', 'radecTai', 'gPSFluxLinearSlope',
       'gPSFluxLinearIntercept', 'gPSFluxMAD', 'gPSFluxMaxSlope',
       'gPSFluxErrMean', 'gPSFluxMean', 'gPSFluxMeanErr', 'gPSFluxNdata',
       'gTOTFluxMean', 'gTOTFluxMeanErr', 'gPSFluxMin', 'gPSFluxMax',
       'gPSFluxPercentile05', 'gPSFluxPercentile25',
       'gPSFluxPercentile50', 'gPSFluxPercentile75',
       'gPSFluxPercentile95', 'gPSFluxSigma', 'gTOTFluxSigma',
       'gPSFluxSkew', 'gPSFluxChi2', 'gPSFluxStetsonJ',
       'rPSFluxLinearSlope', 'rPSFluxLinearIntercept', 'rPSFluxMAD',
       'rPSFluxMaxSlope', 'rPSFluxErrMean', 'rPSFluxMean',
       'rPSFluxMeanErr', 'rPSFluxNdata', 'rTOTFluxMean',
       'rTOTFluxMeanErr', 'rPSFluxMin', 'rPSFluxMax',
       'rPSFluxPercentile05', 'rPSFluxPercentile25',
       'rPSFluxPercentile50', 'rPSFluxPercentile75',
       'rPSFluxPercentile95', 'rPSFluxSigma', 'rTOTFluxSigma',
       'rPSFluxSkew', 'rPSFluxChi2', 'rPSFluxStetsonJ',
       'iPSFluxLinearSlope', 'i

In [10]:
# identify some diaobjects with lots of epochs
filters = ['g','r','i','z','y']
objTable.loc[:,[f'{filt}PSFluxNdata' for filt in filters]].apply(np.sum,axis=1).sort_values()

diaObjectId
3425299777917552444     0.0
3425273389638496005     0.0
3425392136894297560     0.0
3425282185731513377     0.0
3425502088057069082     0.0
                       ... 
3425370146661728262    38.0
3425352554475684013    39.0
3425352554475684103    39.0
3425282185731506409    39.0
3425348156429172907    41.0
Length: 949028, dtype: float64

## Source tables

In [11]:
refs = sorted(registry.queryDatasets("diaSourceTable_tract"))

In [12]:
for ref in refs: print(ref.dataId.full)

{skymap: 'hsc_rings_v1', tract: 9615}
{skymap: 'hsc_rings_v1', tract: 9697}
{skymap: 'hsc_rings_v1', tract: 9813}


In [13]:
# this is an extremeley large table, so let's load just a handful of columns:
lc_cols = ['filterName','diaObjectId','midPointTai','ra','decl','psFlux','psFluxErr']
sourceTable = butler.get(refs[0],parameters={'columns':lc_cols})
sourceTable

,filterName,diaObjectId,midPointTai,ra,decl,psFlux,psFluxErr
diaSourceId,,,,,,,
201880642781390,y,3425264593545462643,56741.629032,216.960597,-0.073707,-1191.955816,243.198961
569529843319232,i,3425264593545462643,56744.616684,216.960631,-0.073730,119.179375,61.713948
10270155862966957,r,3425264593545462643,57099.633578,216.960624,-0.073768,544.873854,50.054954
549826680848770,i,3425264593545462643,56744.545180,216.960553,-0.073701,-349.268988,46.007548
10251343906210458,r,3425264593545462643,57099.589358,216.960628,-0.073753,593.962955,47.595019
...,...,...,...,...,...,...,...
11610333835690094,y,3425616437266356005,57110.428346,215.616441,1.564203,2623.135703,247.826403
11610333835690095,y,3425616437266356006,57110.428346,215.620109,1.579734,6057.184253,325.333169
11610333835690096,y,3425616437266356007,57110.428346,215.618651,1.564250,1452.406183,249.545325


In [14]:
sourceTable.columns

Index(['filterName', 'diaObjectId', 'midPointTai', 'ra', 'decl', 'psFlux',
       'psFluxErr'],
      dtype='object')

Let's find a DIAObject with lots of detections

In [70]:
count = sourceTable.iloc[:100000].groupby('diaObjectId').agg(len)

In [79]:
count.iloc[count['filterName'].argmax()]

filterName     39
midPointTai    39
ra             39
decl           39
psFlux         39
psFluxErr      39
Name: 3425282185731506409, dtype: int64

In [80]:
test_DiaObjectId = 3425264593545461819
wt = sourceTable.diaObjectId == test_DiaObjectId
sourceTable[wt]

,filterName,diaObjectId,midPointTai,ra,decl,psFlux,psFluxErr
diaSourceId,,,,,,,
201880642781451,y,3425264593545461819,56741.629032,217.029675,-0.061620,2822.906780,293.853234
569529843319375,i,3425264593545461819,56744.616684,217.029689,-0.061624,7699.329790,114.883803
11663430368887032,z,3425264593545461819,57110.626265,217.029698,-0.061606,38968.855922,265.350616
199310104854790,y,3425264593545461819,56741.620087,217.029715,-0.061674,2957.931498,298.177432
565241318474295,i,3425264593545461819,56744.599638,217.029712,-0.061630,183.087372,101.779247
11659141844041959,z,3425264593545461819,57110.612563,217.029708,-0.061622,37656.748296,260.693140
10270155862967070,r,3425264593545461819,57099.633578,217.029729,-0.061607,-691.137106,132.155182
11197868798902916,g,3425264593545461819,57106.611946,217.029727,-0.061637,660.285803,104.062677
203613662085447,y,3425264593545461819,56741.635250,217.029705,-0.061637,2609.472365,301.339895


In [ ]:
# a nontrivial example would be: compute max dm/dt per (diaObjectId, band)

In [54]:
from lsst.analysis.tools.interfaces import AnalysisTool
from lsst.pex.config import Field

from lsst.analysis.tools.actions.scalar.scalarActions import CountAction, FracThreshold, MedianAction
from lsst.analysis.tools.actions.vector import (
    BandSelector,
    LoadVector,
    MultiCriteriaDownselectVector,
    PerGroupStatistic,
    ThresholdSelector,
)

class DiaSourceRange(AnalysisTool):
    """Example AnalysisTool which computes the max-min DiaSource flux.
    
    adapted in part from StellarPhotometricRepeatability
       
    """
#    Compute photometric repeatability from multiple measurements of a set of
#    stars. First, a set of per-source quality criteria are applied. Second,
#    the individual source measurements are grouped together by object index
#    and per-group quantities are computed (e.g., a representative S/N for the
#    group based on the median of associated per-source measurements). Third,
#    additional per-group criteria are applied. Fourth, summary statistics are
#    computed for the filtered groups

    fluxType = Field[str](doc="Flux type to calculate range with", default="psFlux")

    def setDefaults(self):
        super().setDefaults()

        # Apply per-source selection criteria
        self.prep.selectors.bandSelector = BandSelector()
        self.prep.selectors.bandSelector.vectorKey  = 'band'
        self.prep.selectors.bandSelector.bands = ['y']
        print(self.prep)

        # Compute per-group quantities
        self.process.buildActions.perGroupRange = PerGroupStatistic()
        self.process.buildActions.perGroupRange.buildAction.vectorKey = f"{self.fluxType}"
        self.process.buildActions.perGroupRange.func = "median"
        self.process.buildActions.perGroupRange.groupKey = "diaObjectId"

        self.process.buildActions.perGroupCount = PerGroupStatistic()
        self.process.buildActions.perGroupCount.buildAction.vectorKey = f"{self.fluxType}"
        self.process.buildActions.perGroupCount.func = "count"
        self.process.buildActions.perGroupCount.groupKey = "diaObjectId"


 #       # Filter on per-group quantities
 #       self.process.filterActions.perGroupRangeFiltered = MultiCriteriaDownselectVector(
 #           vectorKey="perGroupRange"
 #       )
 #       # require at least two detections
 ##       self.process.filterActions.perGroupRangeFiltered.selectors.count = ThresholdSelector(
  #          vectorKey="perGroupCount",
  #          op="ge",
  #          threshold=2,
  #      )

        # Compute summary statistics on filtered groups
        self.process.calculateActions.diaSourceRange = MedianAction(vectorKey="perGroupRange")
        self.process.calculateActions.diaSourceRangeNsources = CountAction(vectorKey="perGroupRange")

        self.produce.metric.units = {  # type: ignore
            "diaSourceRange": "nJy",
            "diaSourceRangeNsources": "ct",
        }
        self.produce.metric.newNames = {
            "diaSourceRange": "{band}_diaSourceRange",
            "diaSourceRangeNsources": "{band}_ct",
        }

In [36]:
bs = BandSelector()
bs.vectorKey  = 'filterName'
mask = bs(sourceTable.iloc[:1000])
print(np.sum(mask))

1000


In [16]:
testSourceTable = sourceTable.iloc[:1000]

In [17]:
wct_gt_2 = testSourceTable.loc[:,['diaObjectId','filterName','midPointTai']].groupby(
    ['diaObjectId','filterName']).agg(len).apply(lambda x: x >=2)

In [18]:
wct_gt_2

midPointTai
diaObjectId         filterName             
3425264593545461784 y                  True
3425264593545461789 i                  True
                    r                 False
                    y                  True
3425264593545461790 g                  True
...                                     ...
3425264593545462724 y                 False
3425264593545462725 y                 False
3425264593545462726 y                 False
3425264593545462727 y                 False
3425264593545462728 y                 False

[302 rows x 1 columns]

In [21]:
miniTest = testSourceTable.loc[(testSourceTable.diaObjectId == 3425264593545461784) & (testSourceTable.filterName=='y'),:] 

In [23]:
miniTest

,filterName,diaObjectId,midPointTai,ra,decl,psFlux,psFluxErr
diaSourceId,,,,,,,
201880642781497,y,3425264593545461784,56741.629032,217.074033,-0.063161,-2387.114386,275.227595
183034326286375,y,3425264593545461784,56741.557356,217.074031,-0.063203,-2174.459202,281.279721
202827683070132,y,3425264593545461784,56741.632377,217.074029,-0.063171,3986.462926,516.121906
165042708283585,y,3425264593545461784,56741.506686,217.074041,-0.063182,-2550.321189,390.453978
166773580103795,y,3425264593545461784,56741.510958,217.074082,-0.063183,-2579.586333,298.919255
11625812897825095,y,3425264593545461784,57110.477642,217.074037,-0.063175,-2345.908411,224.049678


In [32]:
%pdb

Automatic pdb calling has been turned OFF


In [58]:
atool = DiaSourceRange()
atool.finalize()

{'vectorKeys': [], 'selectors': {'bandSelector': {'vectorKey': 'band', 'bands': ['y']}}}


In [59]:
atool.prep

lsst.analysis.tools.interfaces._stages.BasePrep(vectorKeys=['psFlux', 'diaObjectId'], selectors={'bandSelector': {'vectorKey': 'band', 'bands': ['y']}})

In [60]:
atool.prep(miniTest.rename(columns={'filterName':'band'}))

{'psFlux': diaSourceId
 201880642781497     -2387.114386
 183034326286375     -2174.459202
 202827683070132      3986.462926
 165042708283585     -2550.321189
 166773580103795     -2579.586333
 11625812897825095   -2345.908411
 Name: psFlux, dtype: float64,
 'diaObjectId': diaSourceId
 201880642781497      3425264593545461784
 183034326286375      3425264593545461784
 202827683070132      3425264593545461784
 165042708283585      3425264593545461784
 166773580103795      3425264593545461784
 11625812897825095    3425264593545461784
 Name: diaObjectId, dtype: int64}

In [63]:
atool = DiaSourceRange()
atool.finalize()
metric = atool(miniTest.rename(columns={'filterName':'band'}))


metric

{'vectorKeys': [], 'selectors': {'bandSelector': {'vectorKey': 'band', 'bands': ['y']}}}


{'analysisTools_diaSourceRange': Measurement('analysisTools_diaSourceRange', <Quantity -2366.51139849 nJy>, notes={'analysisTools_diaSourceRange.metric_tags': []}),
 'analysisTools_ct': Measurement('analysisTools_ct', <Quantity 1. ct>, notes={'analysisTools_ct.metric_tags': []})}

In [113]:
data  = {}
for col in sourceTable.iloc[:1000].columns:
    data[col] = sourceTable.iloc[:1000][col].values